In [1]:
#import pickle
import numpy as np
import pandas as pd
#import Bio
from Bio import AlignIO
import os, urllib, gzip, re

data_dir='data_test'

In [2]:
files = ['Pfam-A.full.gz']

In [3]:
pfam_file = os.path.join(data_dir, 'pfam.npy')

pfam = []
with gzip.open(os.path.join(data_dir, 'Pfam-A.full.gz'), 'r') as f:
    for i, line in enumerate(f):
        if line[:7] == '#=GF AC':
            
            ## name of PF (e.g., PF10417)
            ac = line.split(' ')[4][:-1].split('.')[0]            
            pfam.append([ac, 0, 0, 0]) 
            #pfam.append(ac)
      
        # 2018.12.03: test
        #pfam.append(line)
        if i == 50000: break        
            
pfam = np.array(pfam)
n_pfam = len(pfam)

#print(pfam.shape)
#print(pfam)

In [4]:
# parse multiple sequence alignments file with Biopython
alignments = AlignIO.parse(gzip.open(os.path.join(data_dir, 'Pfam-A.full.gz'), 'r'),'stockholm')

In [5]:
# for each pfam/msa
for i, a in enumerate(alignments):
    
    # test with the first pfam in the downloaded data (PF10417)
    if i > 0: break   
    
    # local directory associated with pfam
    pfam_dir = os.path.join(data_dir, 'Pfam-A.full', pfam[i, 0])

    try:
        os.makedirs(pfam_dir)
    except:
        pass
            
    # number of residues/sequences in alignment
    n_residue = a.get_alignment_length()    
    n_sequence = len(a)
    
    #print('n_residue,n_sequence:')
    #print(n_residue,n_sequence)

    #------------------------------------------------------------------
    # msa: residues symbols
    # pdb_refs: references to pdb
    msa = np.empty((n_residue, n_sequence), dtype=str)
    pdb_refs = []
    
    # for each sequence in alignment
    # 'a' is a BioPython object that corresponds to a list of sequences in an MSA, 
    # this will iterate through the sequences            
    for j, seq in enumerate(a):
        # store residue symbols in lowercase
        # The actual list of residues is in the 'seq' member variable of 'seq', 
        # this converts those letters to lowercase
        #msa[:, j] = np.array(seq.seq.lower())
        msa[:, j] = np.array(seq.seq)  # 2018.12.03: Tai
    
        # msa[:,j] is aligned sequence j (or t in Tai's notation) 
        #print('msa[:,j]')
        #print(msa[:, j])
        
        # store uniprot sequence id. This splits 'seq_id' on either '/' or '-' characters, 
        # so that 'id-0/100' splits into ['id', '0', '100']
        uniprot_id, uniprot_start, uniprot_end = re.split('[/-]', seq.id)
        
        # extract pdb refs if they are present. 'dbxrefs' is the member variable of the Biopython
        # object seq that contains pdb cross-reference info
        refs = seq.dbxrefs
        #print(refs)
        
        # if refs is an empty list, i.e. [], then continue to the next seq in the for loop        
        if not refs:
            continue
            
        # for each element in the the list 'refs'            
        for ref in refs:
            #print('ref:')
            #print(ref) # 'PDB; 3KB6 B; 106-296'
            
            # remove white space and split on ';' so that ' PDB;XYZA;0-100 ' becomes ['PDB', 'XYZA', '0-100']                    
            ref = ref.replace(' ', '').split(';')
            if ref[0] == 'PDB':
                # split id and chain info so that pdb_id='XYZ' and chain='A' continuing with the same example                        
                pdb_id, chain = ref[1][:-1], ref[1][-1]
                
                # split '0-100' to ['0', '100']
                pdb_start_end = ref[2].split('-')
                if len(pdb_start_end) == 2:
                    pdb_start, pdb_end = pdb_start_end                    
                # If for some reason pdb_start_end is not a list of length 2, then continue to the next seq, 
                # this is a bit hacky but I found it necessary for a few apparently ill-formatted cross-references    
                else:
                    continue
                pdb_refs.append([pfam[i,0],j-1,uniprot_id,uniprot_start,uniprot_end,pdb_id,chain,pdb_start,pdb_end])
                # pfam[i,0] : pfam id (PF10417), j: sequence h pdb_ref
                
                #print('pdb_refs:')
                #print(pfam[i, 0],j-1,uniprot_id, uniprot_start,uniprot_end,pdb_id,chain,pdb_start,pdb_end)
                
    # save an alignment matrix as a numpy binary for each MSA
    np.save(os.path.join(pfam_dir,'msa.npy'),msa)
    
    # save the pdb cross reference info as a numpy binary for each MSA
    np.save(os.path.join(pfam_dir,'pdb_refs.npy'), pdb_refs)
            
    n_pdb_ref = len(pdb_refs)
    pfam[i, 1:] = n_residue, n_sequence, n_pdb_ref
          
np.save(pfam_file, pfam)        

In [6]:
print(msa[0,:])

['-' '-' '-' ... '-' '-' '-']


In [7]:
# compile all pdb_refs having pdb (n_pdb > 0)
pdb_refs_file = os.path.join(data_dir, 'pdb_refs.npy')
# this file takes a lot of work to generate, so if it is cached already, then load it, else generate and cache it
if os.path.exists(pdb_refs_file):
    print('load')
    pdb_refs = np.load(pdb_refs_file)
else:
    # find pfam has pdb
    # number of pdb
    n_pdb_refs = pfam[:, 3].astype(int)
    has_pdb_refs = n_pdb_refs > 0
    n_pdb_refs = n_pdb_refs[has_pdb_refs]
    pfam_with_pdb = pfam[has_pdb_refs, 0]
    
    print(pfam_with_pdb)
    
    # read pdb_refs.npy from folder of pfam having pdb
    # the first one
    refs = os.path.join(data_dir, 'Pfam-A.full', pfam_with_pdb[0],'pdb_refs.npy')
    pdb_refs = np.load(refs)
    for fam in pfam_with_pdb[1:]:
        print('add the 2nd, 3rd,... to the 1st')        
        refs = np.load(os.path.join(data_dir, 'Pfam-A.full', fam, 'pdb_refs.npy'))
        pdb_refs = np.vstack([pdb_refs, refs])
    np.save(pdb_refs_file, pdb_refs)

load


In [8]:
# convert pfam and pdb_refs to pandas dataframes
pfam = pd.DataFrame(index=pfam[:, 0],data=pfam[:, 1:],columns=['res', 'seq', 'pdb_refs'],dtype=int)
pdb_refs = pd.DataFrame(index=pdb_refs[:, 0],data=pdb_refs[:, 1:],columns=[
        'seq', 'uniprot_id', 'uniprot_start', 'uniprot_end', 'pdb_id', 'chain', 'pdb_start', 'pdb_end'])
cols = ['seq', 'uniprot_start', 'uniprot_end', 'pdb_start', 'pdb_end']
pdb_refs[cols] = pdb_refs[cols].apply(pd.to_numeric)

In [9]:
# remove rows where the length between uniprot start/end is different from pdb start/end
cols = ['uniprot_start', 'uniprot_end', 'pdb_start', 'pdb_end']
start_end = pdb_refs[cols].values
consistent_length = np.diff(start_end[:, :2], axis=1) == np.diff(start_end[:, 2:], axis=1)
pdb_refs = pdb_refs[consistent_length]